In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import unidecode
from lyricsgenius import Genius
import spotipy
import spotipy.util as util
import text2emotion as te

/Users/f/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/text2emotion/__init__.py:2728: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sum(emotions.values()) is 0:
[nltk_data] Downloading package stopwords to /Users/f/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/f/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/f/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
env_path = find_dotenv()
load_dotenv(env_path)

True

# Testing Genius API

In [3]:
# loading needed credentials
genius_token = os.environ.get('LYRICSGENIUS_ACCESS_TOKEN')

In [16]:
# instantiate the class with some parameters
genius = Genius(genius_token, 
                timeout=120, 
                remove_section_headers=True, 
                sleep_time=0.3, 
                skip_non_songs=True)

# search with the user input (artist + song title)
user_input_artist = "nothing nowhere"
user_input_song = "rejecter"


song = genius.search_song(title = user_input_song, artist = user_input_artist)

# getting song id
#song_id = song['sections'][0]['hits'][0]['result']['id']
#song_id

Searching for "rejecter" by nothing nowhere...
Done.


In [17]:
song.to_dict()['full_title']


'\u200brejecter by\xa0\u200bnothing,nowhere.'

In [19]:
test_api_response = unidecode.unidecode(song.to_dict()['full_title'].replace('\xa0', ' ').replace('\u200b', ' ').lower())
test_api_response = test_api_response.split(' by ')

In [7]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [20]:
song_similar = similar(test_api_response[0], unidecode.unidecode(user_input_song.lower()))
artist_similar = similar(test_api_response[1], unidecode.unidecode(user_input_artist.lower()))
print(song_similar)
print(artist_similar)

0.9411764705882353
0.875


In [25]:
if song_similar > 0.9:
    
    # all lyrics responses come with the song's title and 'Lyrics' str
    characters_to_remove = len(song.to_dict()['title'] + ' Lyrics')   
    
    # 
    lyrics = genius.lyrics(song.to_dict()['id'])[characters_to_remove:-5]
    
    print(lyrics)
    
else:
    
    lyrics = None
    
    print(lyrics)


You cannot hide
And there's no sign
The slow decline
In your mind
You are not mine
I'm not yours
Why would I try
When you're not sure?
I've been falling down, upstate
Try to save me, it's too late
Voices in my head, talking
So I got the keys, I hop in
Like I am just a reject
Putting back the pieces
Pushing you away like
I don't even need this
Laying on the freeway
Thinking 'bout what she say
Waiting for a semi-truck
Like I don't even give a fuck
I just need a reason
I can't find a reason
Watching all the cars go
Passing like the seasons
Feeding all my demons
See you when I'm sleeping
Now I think I like it like the pain became my Jesus

Ooh, everything fade when I get this way
Ooh, every damn day gotta feel like this
Ooh, everyone knows that I lost my way
Ooh, every damn day gonna be the same

There's nothing in me
Like asphalt or concrete
Memories, fifteen
Suffocating in my sheets
I can hear the sound, racing
Laying down now, take me
Something in my head, hiding
I see the lights, shin

In [26]:
emotions = te.get_emotion(lyrics)
emotions

{'Happy': 0.19, 'Angry': 0.06, 'Surprise': 0.4, 'Sad': 0.21, 'Fear': 0.14}

In [14]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /Users/f/nltk_data...


In [24]:
sid.polarity_scores(lyrics)

{'neg': 0.132, 'neu': 0.732, 'pos': 0.136, 'compound': -0.0372}

In [15]:
# all lyrics responses come with the song's title and 'Lyrics' str
characters_to_remove = len(
    song['sections'][0]['hits'][0]['result']['title'] + ' Lyrics')

In [16]:
# getting lyrics
lyrics = genius.lyrics(song_id)[characters_to_remove:-5]
lyrics

"\n\nSo close, no matter how far\nCouldn't be much more from the heart\nForever trusting who we are\n\nAnd nothing else matters\n\nNever opened myself this way\nLife is ours, we live it our way\nAll these words I don't just say\nAnd nothing else matters\n\nTrust I seek and I find in you\nEveryday, for us, something new\nOpen mind for a different view\n\nAnd nothing else matters\n\nNever cared for what they do\nNever cared for what they know\nBut I know\n\nSo close, no matter how far\nIt couldn't be much more from the heart\nForever trusting who we are\n\nAnd nothing else matters\n\nNever cared for what they do\nNever cared for what they know\nBut I know\nYou might also like\n\nI never opened myself this way\nLife is ours, we live it our way\nAll these words I don't just say\n\nAnd nothing else matters\n\nTrust I seek and I find in you\nEveryday, for us, something new\nOpen mind for a different view\n\nAnd nothing else matters\n\nNever cared for what they say\nNever cared for games they

In [17]:
# simple creanup for text2emotion testing
cleaned_lyrics = lyrics.replace('\n', ' ')
cleaned_lyrics

"  So close, no matter how far Couldn't be much more from the heart Forever trusting who we are  And nothing else matters  Never opened myself this way Life is ours, we live it our way All these words I don't just say And nothing else matters  Trust I seek and I find in you Everyday, for us, something new Open mind for a different view  And nothing else matters  Never cared for what they do Never cared for what they know But I know  So close, no matter how far It couldn't be much more from the heart Forever trusting who we are  And nothing else matters  Never cared for what they do Never cared for what they know But I know You might also like  I never opened myself this way Life is ours, we live it our way All these words I don't just say  And nothing else matters  Trust I seek and I find in you Everyday, for us, something new Open mind for a different view  And nothing else matters  Never cared for what they say Never cared for games they play  Never cared for what they do Never cared

# Testing Spotify API

In [5]:
# needed credentials and permissions (scope)
cid = os.environ.get('SPOTIFY_CLIENT_ID')
secret = os.environ.get('SPOTIFY_CLIENT_SECRET')
redirect_uri = os.environ.get('SPOTIPY_REDIRECT_URI')
scope = 'playlist-modify-private'
username = None

In [6]:
# testing locally for user authentication, later development is needed
spotify_token = util.prompt_for_user_token(username,
                                           scope,
                                           cid,
                                           secret,
                                           redirect_uri)

sp = spotipy.Spotify(auth=spotify_token)

In [7]:
# getting user id
user_id = sp.me()['id']

In [8]:
# creating a private playlist
playlist = sp.user_playlist_create(user_id, 
                                   'test', 
                                   public=False, 
                                   collaborative=False, 
                                   description='Testing')

In [9]:
# obtaining playlist id
playlist_id = playlist['id']
playlist_id

'1Q71YBTedi64Hy1gnUe5Tg'

In [10]:
# artist URI hardcoded for testing purposes
# this top_tracks_list will be replaced by the IDs 
# extracted by the recommender
top_tracks = sp.artist_top_tracks('spotify:artist:2NFrAuh8RQdQoS7iYFbckw')

In [11]:
# get a list of tracks URI
top_tracks_list = [tracks['uri'] for tracks in top_tracks['tracks']]
top_tracks_list

['spotify:track:4loXMor75kKVBB03ygwDlh',
 'spotify:track:4PMmwowVLOajPdiKnrU1vK',
 'spotify:track:3nNNYRBtkNsKjertmtTRMO',
 'spotify:track:6tJPdGKrbAeyhkkhn44RHR',
 'spotify:track:6S10TQTWVy1Zf3bscMFBlU',
 'spotify:track:1K0urJwRWgKsdufGEdkXP7',
 'spotify:track:24B4TFlEOJzzqb33L4sbI5',
 'spotify:track:4um6CPDIxnNWSEbj3LJQhQ',
 'spotify:track:5s9eoyEyoeEj3fqtt0pQoQ',
 'spotify:track:2hj7EbKUGH0xtAzv2q22dZ']

In [12]:
# add songs to playlist
sp.user_playlist_add_tracks(username, 
                            playlist_id, 
                            top_tracks_list, 
                            position=None)

{'snapshot_id': 'MixhNDEzMzEzMTdmNTExMjNlYzlmMTMwMDg5M2NkNDY3OGU1YzAwNWIz'}